In [150]:
using DataFrames
using CSV
ENV["COLUMNS"] = 1000
using CategoricalArrays


using Pipe: @pipe
using Setfield

## Initial steps

In [38]:
df = CSV.File("data/train.csv") |> DataFrame

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Int64,Int64,String,String7,Float64?,Int64,Int64,String31,Float64,String15?,String1?
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,missing,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,missing,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,missing,S
6,6,0,3,"Moran, Mr. James",male,missing,0,0,330877,8.4583,missing,Q
7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,missing,S
9,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,missing,S


In [22]:
names(df) |> show
describe(df) 

["PassengerId", "Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,PassengerId,446.0,1,446.0,891,0,Int64
2,Survived,0.383838,0,0.0,1,0,Int64
3,Pclass,2.30864,1,3.0,3,0,Int64
4,Name,,"Abbing, Mr. Anthony",,"van Melkebeke, Mr. Philemon",0,String
5,Sex,,female,,male,0,String7
6,Age,29.6991,0.42,28.0,80.0,177,"Union{Missing, Float64}"
7,SibSp,0.523008,0,0.0,8,0,Int64
8,Parch,0.381594,0,0.0,6,0,Int64
9,Ticket,,110152,,WE/P 5735,0,String31


In [23]:
df[1:5,[:Survived,:Name]]

,Survived,Name
,Int64,String
1,0,"Braund, Mr. Owen Harris"
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"
3,1,"Heikkinen, Miss. Laina"
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
5,0,"Allen, Mr. William Henry"


In [43]:
# List of recorded sexes
levels(df[!,:Sex])

2-element Vector{Bool}:
 0
 1

In [42]:
# Convert Survived column to boolean
df[!,:Survived] = convert.(Bool,df[!,:Survived])

# Convert Sex column to boolean
eltype(df[!,:Sex]) != Bool && (df[!,:Sex] = df[!,:Sex] .== "male")

df.Embarked = categorical(df.Embarked)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Bool,Int64,String,Bool,Float64?,Int64,Int64,String31,Float64,String15?,String1?
1,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.25,missing,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",0,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.925,missing,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.05,missing,S
6,6,0,3,"Moran, Mr. James",1,missing,0,0,330877,8.4583,missing,Q
7,7,0,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,E46,S
8,8,0,3,"Palsson, Master. Gosta Leonard",1,2.0,3,1,349909,21.075,missing,S
9,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.0,0,2,347742,11.1333,missing,S


In [194]:
# Get missing stats

emptycols = count.(ismissing, eachcol(df))
emptyperc = round.(emptycols ./ fill(size(df)...), digits = 3)
sort(DataFrame(Column = names(df), Total = emptycols, Percent = emptyperc), :Total, rev = true)

,Column,Total,Percent
,String,Int64,Float64
1,Cabin,687,0.771
2,Age,177,0.199
3,Embarked,2,0.002
4,PassengerId,0,0.0
5,Survived,0,0.0
6,Pclass,0,0.0
7,Name,0,0.0
8,Sex,0,0.0
9,SibSp,0,0.0


In [122]:

colwise(ismissing,df)
# count.(ismissing, eachcol(df))

LoadError: UndefVarError: colwise not defined

In [192]:
cabin = @pipe dropmissing(df[:,[:Survived,:Cabin]]) |> sort(_,rev=true)
show(cabin, allrows=true)
cabin.Cabin = getindex.(cabin[:,:Cabin],1)

204×2 DataFrame
 Row │ Survived  Cabin           
     │ Bool      String15        
─────┼───────────────────────────
   1 │     true  G6
   2 │     true  G6
   3 │     true  F4
   4 │     true  F4
   5 │     true  F33
   6 │     true  F33
   7 │     true  F33
   8 │     true  F2
   9 │     true  F2
  10 │     true  F E69
  11 │     true  E8
  12 │     true  E8
  13 │     true  E68
  14 │     true  E67
  15 │     true  E50
  16 │     true  E49
  17 │     true  E44
  18 │     true  E40
  19 │     true  E36
  20 │     true  E34
  21 │     true  E33
  22 │     true  E33
  23 │     true  E25
  24 │     true  E25
  25 │     true  E24
  26 │     true  E24
  27 │     true  E17
  28 │     true  E121
  29 │     true  E121
  30 │     true  E12
  31 │     true  E101
  32 │     true  E101
  33 │     true  E101
  34 │     true  E10
  35 │     true  D9
  36 │     true  D7
  37 │     true  D56
  38 │     true  D49
  39 │     true  D47
  40 │     true  D45
  41 │     true  D37
  42 │     true  D36
  4

204-element Vector{Char}:
 'G': ASCII/Unicode U+0047 (category Lu: Letter, uppercase)
 'G': ASCII/Unicode U+0047 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 ⋮
 'B': ASCII/Unicode U+0042 (category Lu: Letter, uppercase)
 'B': ASCII/Unicode U+0042 (category Lu: Letter, uppercase)
 'B': ASCII/Unicode U+0042 (category Lu: Letter, uppercase)
 'B': ASCII

In [191]:
@pipe groupby(cabin, :Cabin) |>
      combine(_,
          :Survived => sum => :Survived,
          :Survived => (s -> count(s .== false)) => :Died,
          nrow,
          :Survived => (s -> round(sum(s) / count(s .== false), digits = 2)) => :Ratio
      ) |> sort(_, :Ratio, rev = true)

,Cabin,Survived,Died,nrow,Ratio
,Char,Int64,Int64,Int64,Float64
1,D,25,8,33,3.12
2,E,24,8,32,3.0
3,B,35,12,47,2.92
4,F,8,5,13,1.6
5,C,35,24,59,1.46
6,G,2,2,4,1.0
7,A,7,8,15,0.88
8,T,0,1,1,0.0
